### import bibliotek

In [10]:
import requests
from bs4 import BeautifulSoup
import json
import os

Narzedzia


In [11]:
def extract(ancestor, selector=None, attribute=None, many=False):
    if selector:
        if many:
            if attribute:

                return [item[attribute].strip() for item in ancestor.select(selector)]

            return [item.text.strip() for item in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        try:
            
            return ancestor[attribute]
        except TypeError:
            return None
    return ancestor.text.strip()

In [12]:
opinion_schema={
    'review_id':  (None, 'data-entry-id',),
    'author': ( 'span.user-post__author-name',),
    'recomendation': ('span.user-post__author-recomendation > em',),
    'stars': ( 'span.user-post__score-count',),
    'content': ('div.user-post__text',),
    'pros': ( 'div.review-feature__item--positive', None, True),
    'cons': ( 'div.review-feature__item--negative', None, True),
    'likes': ( 'button.vote-yes > span',),
    'dislikes': ( 'button.vote-no > span',),
    'publish_date': ( 'span.user-post__published > time:nth-child(1)', 'datetime'),
    'purchase_date': ('span.user-post__published > time:nth-child(2)', 'datetime')
    }

In [13]:
with open('./headers.json', 'r') as jf:

    headers=json.load(jf)


Pobranie ze strony Ceneo.pl opinii o  konkretnym produkcie



In [ ]:
product_id=input('Podaj kod  productu: ')
next_page=f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinios=[]
while next_page:
    response = requests.get(next_page, headers = headers)
    print(next_page)
    if response.status_code==200:
        page_dom=  BeautifulSoup(response.text, "html.parser")
        opinions= page_dom.select('div.js_product-review:not(.user-post--highlight)')
        print(len(opinions))
        for opinion in opinions:

                single_opinion={
                    key: extract(opinion, *value)
                    for key, value in opinion_schema.items()
                }
                all_opinios.append(single_opinion)
    
        try:
            next_page="https://www.ceneo.pl"+extract(page_dom, 'a.pagination__next', 'href')
                
        except TypeError:
            next_page=None
print(all_opinios)

### zapisanie wszystkich opinii w bazie danych


In [15]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [16]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinios, jf, indent= 4, ensure_ascii=False)